In [ ]:
from os.path import join, abspath, expanduser
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import datetime
from rio_geom import rio_to_exterior
from datetime import timedelta
import geopandas as gpd
import rioxarray as rxa
import contextily as ctx
from tqdm import tqdm
import seaborn as sns
from uavsar_pytools.snow_depth_inversion import depth_from_phase

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
data_dir = expanduser('~/scratch/data/uavsar/snowpits')
pits_dir = join(data_dir, 'pits')
swe = pd.read_csv(join(data_dir, 'SNEX20_TS_SP_Summary_SWE_v01.csv'), parse_dates = ['Date/Local Standard Time'], index_col= 'PitID')
env = pd.read_csv(join(data_dir, 'SNEX20_TS_SP_Summary_Environment_v01.csv'), parse_dates = ['Date/Local Standard Time'], index_col= 'PitID')

pit_sum = pd.concat([swe, env], axis = 1).reindex(swe.index)
pit_sum = pit_sum.loc[:,~pit_sum.columns.duplicated()]
pit_sum = pit_sum.rename(columns = {'Date/Local Standard Time':'date'})
pit_sum = gpd.GeoDataFrame(pit_sum, geometry=gpd.points_from_xy(pit_sum['Longitude (deg)'], pit_sum['Latitude (deg)']), crs = 4326)

In [ ]:
with open(expanduser('~/scratch/data/uavsar/image_fps'), 'rb') as f:
    image_fps = pickle.load(f)

res = []
for i, image_fp in tqdm(enumerate(image_fps), total = len(image_fps)):
    if i == i:
        if image_fp['flight1'].date().year == 2021 or image_fp['pol'] != 'VV':
            pass
        else:
            dic = {}
            dic['image_data'] = image_fp
            loc = image_fp['location']
            dic['loc'] = loc
            dic['pol'] = image_fp['pol']
            geom = rio_to_exterior(image_fp['cor'])
            for i, dt in enumerate(['flight1', 'flight2']):
                dic[f't{i+1}'] = image_fp[dt]
            # Form a date range to query on either side of our chosen day 
            df_range = pit_sum[(pit_sum.date.dt.date > image_fp['flight1'].date()-pd.Timedelta('1 day')) & (pit_sum.date.dt.date < image_fp['flight2'].date() + pd.Timedelta('1 day'))]

            if len(df_range) > 0:
                # View snow pits that are +/- 1 day of the first UAVSAR flight date
                geom = geom.to_crs(df_range.crs)
                points_within = gpd.sjoin(df_range, geom, predicate='within')
                points_within = points_within.drop(['index_right'], axis=1)
                if len(points_within) > 0:
                    epsg = 26900 + int(points_within.iloc[0]['UTM Zone'].replace('N',''))
                    points_within = points_within.to_crs(epsg = epsg)
                    if len(points_within) > 0:
                        for label in ['fp','inc','cor','hgt']:
                            img = rxa.open_rasterio(image_fp[label])
                            for i_site, r_site in points_within.iterrows():
                                img = img.rio.reproject(dst_crs = epsg)
                                site_name = r_site.Location + '-' + r_site.Site + '-' + str(r_site['date'].date()) + '-' + str(r_site['date'].date())
                                if label == 'fp':
                                    label = 'unw'
                                points_within.loc[i_site, label] = img.sel(x = r_site.geometry.x, y = r_site.geometry.y, method = 'nearest', tolerance = 50).values[0]
                        dic[f'pit_num'] = len(points_within)
                        dic[f'pits'] = points_within
            else:
                dic['pit_num'] = len(points_within)
                print(f'{dt}_{loc} has none')
            res.append(pd.DataFrame.from_dict([dic]))
res = pd.concat(res, ignore_index = True)

In [ ]:
for i, r in res[res.pit_num > 0].iterrows():
    for pit_id, snowpit_results in r.pits.iterrows():
        pass

In [ ]:
r.pits